# LDA Model Training

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Data

In [23]:
data_file_path='DATA.csv'
data=pd.read_csv(data_file_path)
data

,seq_no.,seq,module,motif_1,motif_2,motif_3,motif_4,motif_5,motif_string
0,0,GGAGGAGGAAGAGGCTGGGCCCCTGCTGTGTGGGGGCAAGTTCCCA...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
1,1,CAAATACCCTGGGGTGCAATACGACTTATATCTCACGTATTGGAAG...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
2,2,AACTAGGACACAGAAGTTGATCTAACGTAAACATCAAGAGCTTCCT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
3,3,CACAGCTGGGCCTGGTTGGTCTTTGTCCAGGGAACAATGGAGCGCC...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
4,4,TTGTTTTATTTGTTTGTTGGGGGGCGGCGGGGAGCGACAGGGGAGT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
...,...,...,...,...,...,...,...,...,...
995,995,CTATTATTAAGAAATATACACAATTTTAACTTCAAATATCTCTCAT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3"
996,996,ATTGATTCTCACTTGCTTGACTCAAGGGAGGGTTTGATTTTGGTCA...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
997,997,ATGTGGTTCTACCATATAGTTTATCAATTTTAAACAGGTAAAATAT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4"
998,998,ATTTTGTTGGTTAGGTGATGGAAGTATGATGCTATTGATATTTCCC...,2,0.0,1.0,0.0,0.0,1.0,"motif_2,motif_5"


In [24]:
print(data[data['motif_string'].isna()])
data.dropna(subset=['motif_string'],inplace=True)
data['motif_list']=data['motif_string'].apply(lambda x:x.split(','))
data

Empty DataFrame
Columns: [seq_no., seq, module, motif_1, motif_2, motif_3, motif_4, motif_5, motif_string]
Index: []


,seq_no.,seq,module,motif_1,motif_2,motif_3,motif_4,motif_5,motif_string,motif_list
0,0,GGAGGAGGAAGAGGCTGGGCCCCTGCTGTGTGGGGGCAAGTTCCCA...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
1,1,CAAATACCCTGGGGTGCAATACGACTTATATCTCACGTATTGGAAG...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
2,2,AACTAGGACACAGAAGTTGATCTAACGTAAACATCAAGAGCTTCCT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
3,3,CACAGCTGGGCCTGGTTGGTCTTTGTCCAGGGAACAATGGAGCGCC...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
4,4,TTGTTTTATTTGTTTGTTGGGGGGCGGCGGGGAGCGACAGGGGAGT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
...,...,...,...,...,...,...,...,...,...,...
995,995,CTATTATTAAGAAATATACACAATTTTAACTTCAAATATCTCTCAT...,0,0.0,1.0,1.0,0.0,0.0,"motif_2,motif_3","[motif_2, motif_3]"
996,996,ATTGATTCTCACTTGCTTGACTCAAGGGAGGGTTTGATTTTGGTCA...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
997,997,ATGTGGTTCTACCATATAGTTTATCAATTTTAAACAGGTAAAATAT...,1,1.0,0.0,0.0,1.0,0.0,"motif_1,motif_4","[motif_1, motif_4]"
998,998,ATTTTGTTGGTTAGGTGATGGAAGTATGATGCTATTGATATTTCCC...,2,0.0,1.0,0.0,0.0,1.0,"motif_2,motif_5","[motif_2, motif_5]"


In [25]:
from gensim.corpora import Dictionary

docs=data['motif_list'].values
dictionary=Dictionary(docs)

print('Motif-Index to Motif-Name Mapping:')
for i,v in dictionary.items():
    print(f'{i} - {v}')
    if i==10:
        break
    

2021-02-21 09:05:56,248 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-21 09:05:56,263 : INFO : built Dictionary(5 unique tokens: ['motif_2', 'motif_3', 'motif_1', 'motif_4', 'motif_5']) from 1000 documents (total 1931 corpus positions)


Motif-Index to Motif-Name Mapping:
0 - motif_2
1 - motif_3
2 - motif_1
3 - motif_4
4 - motif_5


In [26]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('BOW (Sequence-0):')
docs[0],corpus[0]

BOW (Sequence-0):


(['motif_2', 'motif_3'], [(0, 1), (1, 1)])

## Training

In [27]:
# #setup logging for trainging metrics 
# import logging
# logging.basicConfig(filename='test_output/model_callbacks.log', filemode='w',
#                     format="%(asctime)s:%(levelname)s:%(message)s",
#                     level=logging.NOTSET)

# from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric
# perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
# convergence_logger = ConvergenceMetric(logger='shell')
# # coherence_cv_logger = CoherenceMetric(corpus=corpus, coherence = 'c_v', texts = docs)

# %%time
# from gensim.models import LdaModel,LdaMulticore

# #HYPERPARAMETERS
# #passes = epochs
# temp = dictionary[0]
# id2word = dictionary.id2token
# lda = LdaModel(corpus, id2word=id2word, alpha='auto',eval_every = 1,\
#                eta='auto',num_topics=3, iterations=5, passes = 10,
#               callbacks=[perplexity_logger,convergence_logger])

# lda.print_topics()

# %%time
# from gensim.models import LdaModel,LdaMulticore

# id2word = dictionary.id2token
# lda = LdaMulticore(corpus, id2word=id2word,eval_every = 1,\
#                eta='auto',num_topics=3, iterations=500, passes = 100)

# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [28]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [29]:
%%time
from gensim.models import LdaModel,LdaMulticore

temp = dictionary[0]
id2word = dictionary.id2token
lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
               eta='auto',num_topics=3, iterations=1000, passes = 5,
              minimum_probability=0.0)

2021-02-21 09:05:56,445 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2021-02-21 09:05:56,446 : INFO : using serial LDA version on this node
2021-02-21 09:05:56,447 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 1000 documents, updating model once every 1000 documents, evaluating perplexity every 1000 documents, iterating 1000x with a convergence threshold of 0.001000
2021-02-21 09:05:56,448 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-02-21 09:05:56,449 : DEBUG : bound: at document #0
2021-02-21 09:05:57,404 : INFO : -2.605 per-word bound, 6.1 perplexity estimate based on a held-out corpus of 1000 documents with 1931 words
2021-02-21 09:05:57,405 : INFO : PROGRESS: pass 0, at document #1000/1000
2021-02-21 09:05:57,406 : DEBUG : performing inference on a chunk of 1000 documents
2021-02-21 09:05:58,129 : DEB

CPU times: user 3.25 s, sys: 42 ms, total: 3.29 s
Wall time: 3.23 s


In [49]:
def getTopicDistribution(predictions,ntopic):
    topic_count_dict={i:0 for i in range(ntopics)}
    ndocs=len(predictions)
    print(ndocs)
    for pred in predictions:
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        topic_count_dict[top_topic]+=1
    topic_dist_dict={k:v/ndocs for k,v in topic_count_dict.items()}
    return topic_dist_dict

def likelihoodMetric(model,corpus):
    likelihood=0
#     P_T=model.alpha
    predictions=lda.get_document_topics(corpus,minimum_probability=0.0)
    P_T=getTopicDistribution(predictions,model.num_topics)
    print(P_T)
    for pred in predictions:
        P_Xi_M=0
        for topic_no,P_Xi_T in pred:
            P_Xi_M+=P_Xi_T*P_T[topic_no]
        likelihood+=np.log10(P_Xi_M)
    print(likelihood)
    return likelihood

In [46]:
from collections import Counter

def findTopMotifs(model,corpus,data,ntopics,ntop=5,outdir=None):
    pred_topic=[]
    for pred in lda.get_document_topics(corpus,minimum_probability=0.0):
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        pred_topic.append(top_topic)
    _data=data.copy()
    _data['pred_topic']=pred_topic      
    gb=_data[['motif_string','pred_topic']].groupby('pred_topic').\
    agg(lambda x: ','.join(x))
    gb['top_motif']=gb['motif_string'].\
    apply(lambda x:Counter(x.split(',')).most_common(ntop))
    gb.reset_index(inplace=True)
    gb=gb[['pred_topic','top_motif']]
    if outdir is not None:
        gb.to_csv(f'{outdir}/top{ntop}_motifs_topics_{ntopics}.csv',index=False)
    print(gb)
    return gb
# findTopMotifs(lda,corpus,data,3)

In [50]:
%%time
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from gensim.models import LdaModel,LdaMulticore

outdir='model_output'
eval_dict={'num_topics':[],'likelihood':[]}
temp = dictionary[0]
id2word = dictionary.id2token
for ntopics in range(2,10):
    print('Num of Topic '+str(ntopics))
    lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
                   eta='auto',num_topics=ntopics, iterations=1000, passes = 5,
                  minimum_probability=0.0)
    print(lda.num_topics)
    print('Finding likelihood...')
    likelihood=likelihoodMetric(lda,corpus)
    print('Finding Top Motifs...')
    findTopMotifs(lda,corpus,data,ntopics,outdir=outdir)
#     print('Findng avg. distance from TSS per topic...')
#     getAvgTssDist(lda,corpus,data,ntopics,outdir=outdir)
    eval_dict['num_topics'].append(ntopics)
    eval_dict['likelihood'].append(likelihood)
eval_df=pd.DataFrame(eval_dict)
eval_df.to_csv(f'{outdir}/likelihood.csv',index=False)
eval_df

Num of Topic 2
2
Finding likelihood...
1000
{0: 0.48, 1: 0.52}
-301.7673207418115
Finding Top Motifs...
   pred_topic                                          top_motif
0           0    [(motif_2, 480), (motif_3, 480), (motif_1, 86)]
1           1  [(motif_1, 478), (motif_4, 325), (motif_5, 42)...
Num of Topic 3
3
Finding likelihood...
1000
{0: 0.042, 1: 0.48, 2: 0.478}
-359.71865087936095
Finding Top Motifs...
   pred_topic                                        top_motif
0           0                   [(motif_5, 42), (motif_2, 40)]
1           1  [(motif_2, 480), (motif_3, 480), (motif_1, 86)]
2           2                 [(motif_1, 478), (motif_4, 325)]
Num of Topic 4
4
Finding likelihood...
1000
{0: 0.325, 1: 0.004, 2: 0.518, 3: 0.153}
-458.08317952349034
Finding Top Motifs...
   pred_topic                                          top_motif
0           0                   [(motif_1, 325), (motif_4, 325)]
1           1         [(motif_1, 4), (motif_2, 4), (motif_3, 4)]
2          

,num_topics,likelihood
0,2,-301.767321
1,3,-359.718651
2,4,-458.083180
3,5,-502.817851
4,6,-466.206306
5,7,-570.154809
6,8,-591.026750
7,9,-557.810497
